# NLP 기초 Assignment
## NLP 제대로 맛보기 - spam.csv를 가지고 유의미한 해석을 도출해주세요!

In [1]:
# 필요한 모듈 불러오기
import pandas as pd

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다. 물론 8주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고, 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다 :)

In [2]:
spam = pd.read_csv('spam.csv')

In [5]:
spam.head(6)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...


In [3]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

## Tokenizing
- 실습코드에서는 한국어 대상이라 konlpy를 다뤘지만, 영어는 nltk 모듈을 쓸 수 있습니다. 의미 있는 토큰으로 쪼개고 불용어도 처리해보세요! 현존하는 불용어 리스트를 사용해도 좋고, 새롭게 만들어도 좋습니다. 특수문자는 없애주고, 모두 소문자로 변환해주는 전처리도 필요하겠죠?

In [39]:
origin_spam_text = spam['v2'].values
origin_spam_text = list(origin_spam_text)
origin_spam_text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

### 소문자로 변환

In [40]:
spam_text = [i.lower() for i in origin_spam_text]
spam_text[0]

'go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...'

### 특수문자 삭제
공백, 숫자, 알파벳대소문자가 아닌 것들은 삭제

In [41]:
import re

spam_text = [re.sub(r"[^a-zA-Z0-9\s]","",i) for i in spam_text]
spam_text[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

### 토큰화

In [44]:
from nltk.tokenize import word_tokenize

nltk.download('punkt')
token_spam_text = [word_tokenize(i) for i in spam_text]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ann99\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
token_spam_text[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

### 불용어 처리

In [46]:
from nltk.corpus import stopwords 

nltk.download('stopwords')
print('영어 불용어 갯수:',len(nltk.corpus.stopwords.words('english')))

영어 불용어 갯수: 179


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ann99\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
stop_words_spam_text = []
stop_words = set(stopwords.words('english')) 

for sentence in token_spam_text:
    result = []
    for token in sentence: 
        if token not in stop_words: 
            result.append(token)
    stop_words_spam_text.append(result)

In [48]:
stop_words_spam_text[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

### 한글자 단어 삭제

In [50]:
for sentence in stop_words_spam_text:
    for word in sentence:
        if len(word) == 1:
            sentence.remove(word)

In [51]:
stop_words_spam_text[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

### Lemmatize
단어원형으로 바꾸기

In [56]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lm = WordNetLemmatizer()

# [[lm.lemmatize(w, pos="v") for w in words] for sentence in stop_words_spam_text]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ann99\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [66]:
let_spam_text = [[lm.lemmatize(w, pos="v") for w in sentence] for sentence in stop_words_spam_text]

In [63]:
origin_spam_text[10]

"I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."

In [67]:
let_spam_text[10]

['im',
 'gon',
 'na',
 'home',
 'soon',
 'dont',
 'want',
 'talk',
 'stuff',
 'anymore',
 'tonight',
 'ive',
 'cry',
 'enough',
 'today']

cried -> cry로 단어의 원형으로 바꿔진 것이 보인다.

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram, GloVe, FastText가 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.